In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [66]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples,silhouette_score, davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox, skew
from sklearn.decomposition import PCA, KernelPCA

In [67]:
def hacForNumClusters(X,n):
    for cluster_num in range(2,n):
        hac(X,cluster_num)
        
def hac(X,cluster_num):
    hac = AgglomerativeClustering(n_clusters = cluster_num)
    labels = hac.fit_predict(X)
    sil = silhouette_score(X, labels)
    db = davies_bouldin_score(X, labels)
    print('clus {}: {}, {}'.format(cluster_num, sil, db))
    return labels
        
def cluster(filepath, num_clus, mult=False):
    df = pd.read_csv(filepath)
    df = df.drop(['Bios'], axis=1)
    
    X = df.to_numpy()
    #minmax scaling
    X_minmax = MinMaxScaler().fit_transform(X)
    
    #dimensionality reduction
    pca = PCA(n_components = 0.99)
    X = pca.fit_transform(X_minmax)
    
    if mult:
        hacForNumClusters(X,num_clus)
    else:
        labels = hac(X,num_clus)
        return labels

In [68]:
print('dataset 1')
labels1 = cluster('org.csv',30,True)


dataset 1
clus 2: 0.13110979245482507, 2.523718796851537
clus 3: 0.11350183171234067, 2.27576431436099
clus 4: 0.14534076344714092, 2.3041647128304996
clus 5: 0.12320377464402425, 2.1213396915745486
clus 6: 0.12751792322631578, 2.020743046093323
clus 7: 0.14704446279755481, 2.077890034739056
clus 8: 0.1631457150382885, 2.084184944744435
clus 9: 0.15048420188500422, 2.1128459750676942
clus 10: 0.14372754804549123, 2.0748823733230566
clus 11: 0.1296749080357924, 2.0773981020704677
clus 12: 0.118329640964038, 2.1589801052345945
clus 13: 0.11731439825403481, 2.2308298722974502
clus 14: 0.11125248853191574, 2.2778935785983645
clus 15: 0.10580318411999988, 2.277235846905591
clus 16: 0.10658404563054132, 2.2952460158294894
clus 17: 0.10665230251282805, 2.2283954064427185
clus 18: 0.10697184244834389, 2.183186921269965
clus 19: 0.10488701755371804, 2.215887428345672
clus 20: 0.10442376431722256, 2.226255295150083
clus 21: 0.10440408669174742, 2.177423202243742
clus 22: 0.10532728064068718, 2.1

In [69]:
print('dataset 1')
labels1 = cluster('org.csv',3)


dataset 1
clus 3: 0.11350183171234067, 2.27576431436099


In [70]:
import os

#os.mkdir(os.getcwd() + '/' + someWord)
def please(filepath, labels, n, someWord):
    df = pd.read_csv(filepath)
    df['cluster #'] = labels
    #os.mkdir(os.getcwd() + '/' + someWord)
    
    
    for cluster in range(n):
        yes = df[df['cluster #'] == cluster]
        filename = '{}/{}.csv'.format(someWord, str(cluster))
        yes.to_csv(filename)
        
    

In [71]:
please('org.csv',labels1,3,'woohoo1')


clustering the clusters

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def clean(filename, n, desc=False):
    df = pd.read_csv(filename)
    text = df['Bios']
    df = "".join([c for c in text if c not in string.punctuation])
    tokenizer = RegexpTokenizer(r'\w+')
    text = text.apply(lambda x: tokenizer.tokenize(x.lower()))
   
    text = [w for w in text if w not in stopwords.words('english')]
    
    # stemmer = PorterStemmer()
    # stem_text = " ".join([stemmer.stem(i) for i in text])
   
    labels = cluster(filename,n,False)
    return labels
    # df['Bios'] = df['Bios'].apply(lambda x: word_lemmatizer(x))
    # stemmer = PorterStemmer()
    # def word_stemmer(text):
    #     stem_text = " ".join([stemmer.stem(i) for i in text])
    #     return stem_text

    # df['Bios'] = df['Bios'].apply(lambda x: word_stemmer(x))
    # print(df.head())
    # #TF-IDF vectorizer
    # tfv = TfidfVectorizer(stop_words = "english",ngram_range = (1,1))
    # #transform
    # vec_text = tfv.fit_transform(df.Bios)
    # #returns a list of words.
    # words = tfv.get_feature_names()

    # kmeans = KMeans(n_clusters = 10, n_init = 17, tol = 0.01, max_iter = 200)
    # #fit the data 
    # kmeans.fit(vec_text)
    # #this loop transforms the numbers back into words
    # common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
    # for num, centroid in enumerate(common_words):
    #     print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

    
# print(clean('./woohoo1/0.csv', 10, True))


In [73]:
def nlp_clus(filename, n, desc=False):
    df = pd.read_csv(filename)
    # Instantiating the Vectorizer, experimenting with both
    vectorizer = CountVectorizer()
    #vectorizer = TfidfVectorizer()

    # Fitting the vectorizer to the Bios
    x = vectorizer.fit_transform(df['Bios'])

    # Creating a new DF that contains the vectorized words
    df_wrds = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names())

    # Concating the words DF with the original DF
    new_df = pd.concat([df, df_wrds], axis=1)

    # Dropping the Bios because it is no longer needed in place of vectorization
    new_df.drop('Bios', axis=1, inplace=True)

    # Instantiating PCA
    pca = PCA()

    # Fitting and Transforming the DF
    df_pca = pca.fit_transform(new_df)

    # Reducing the dataset to the number of features determined before
    pca = PCA(n_components=0.99)

    # Fitting and transforming the dataset to the stated number of features and creating a new DF
    df_pca = pca.fit_transform(new_df)

    # Seeing the variance ratio that still remains after the dataset has been reduced
    print(pca.explained_variance_ratio_.cumsum()[-1])
    
    labels = cluster(filename,n,False)
    return labels

In [74]:
def nlp_cluster_for_cluster_spec(dir_spec):
    dirname = '{}/{}'.format(os.getcwd(),dir_spec)
    for filename in os.listdir(dirname):
        labels = nlp_clus('{}/{}'.format(dirname, filename), 30, True)

In [75]:
nlp_cluster_for_cluster_spec('./woohoo1')

0.9983076104672759
clus 30: 0.14262166959225042, 1.5229368826825644
0.9981959755423443
clus 30: 0.16966916009567212, 1.168422746156556
0.9981118462087376
clus 30: 0.2032303300255138, 1.1252443949886397
1.0


ValueError: Cannot extract more clusters than samples: 30 clusters where given for a tree with 4 leaves.